### Agenda
- Identify the main benefits of Random Forest Algorithm
- Perform Hyperparameter tuning
- Rank Feature by Importance with Random Forests

#### Advantages of Random Forest
- Scaling numerical features is not necessary
- Important features in the data automatically shine through. The algorithm intrinsically takes care of feature selection to a large extent

#### Hyperparameter tuning
- In the previous class we implemented random forests but we used the default arguments. As discussed above there are a lot of parameters that can be modified to fine tune the algorithm even further, and hence improve the performance of the model.
- This fine tuning the parameters of any algorithm is called Hyperparameter tuning. We would use a grid search cross-validation technique to accomplish this. We will discuss the code later.
- Some parameters that can greatly affect the performance of random forests are mentioned below:

```python
n_estimators = [200, 500, 1000, 2000, 4000]
min_samples_split = [2, 4, 8, 16, 32]
min_samples_leaf = [1, 2, 3, 4, 5]
max_features = ['sqrt', 'log2']
max_samples = ['None', 0.5, 0.8]
```

- This is also known as **grid search** as this can also be thought of a grid of possible combinations of all the different values of various parameters that we want to test. Basically the model is tested for each point on the grid and the best combination of values of the parameters is chosen.

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

numerical = pd.read_csv('files_for_lesson_and_activities/numerical.csv')
categorical = pd.read_csv('files_for_lesson_and_activities/categorical.csv')
targets = pd.read_csv('files_for_lesson_and_activities/target.csv')

encoder = OneHotEncoder(drop='first').fit(categorical)
encoded_categorical = encoder.transform(categorical).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)

data = pd.concat([numerical, encoded_categorical, targets], axis = 1)
regression_target = data['TARGET_D']
# data.head()
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

from imblearn.over_sampling import SMOTE
smote = SMOTE()
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis=1)
X_sm, y_sm = smote.fit_resample(X, y)
y_sm.value_counts()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=0)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

#### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150, 200, 500, 1000],
    'min_samples_split': [2, 4],
    'min_samples_leaf' : [1, 2],
    'max_features': ['sqrt']
#    'max_samples' : ['None', 0.5]
    }
clf = RandomForestClassifier(random_state=100)

grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True,n_jobs=-1)
grid_search.fit(X_train,y_train)
grid_search.best_params_ #To check the best set of parameters returned

- Here is a snapshot of the results: [link to the image - Grid Search CV Results](https://education-team-2020.s3-eu-west-1.amazonaws.com/data-analytics/7.09/7.09-grid_search_results.png)

In [ ]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(random_state=0, max_features='sqrt', min_samples_leaf=1, min_samples_split=4, n_estimators=100)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

**Grid Search** is an effective method for adjusting the parameters in supervised learning and improve the generalization performance of a model.</br>
  **RandomizedSearchCV** is very useful when we have many parameters to try and the training time is very long.
  Grid Search is good when we work with a small number of hyperparameters. However, if the number of parameters to consider is particularly high and the magnitudes of influence are imbalanced, the better choice is to use the Random Search.
  [extra help](https://towardsdatascience.com/machine-learning-gridsearchcv-randomizedsearchcv-d36b89231b10)


#### Feature Importance
- It uses impurity-based feature importance measure
- Higher the score, the more important the feature is

In [ ]:
clf.fit( X_train, y_train)
X_train.head()
feature_names = X_train.columns
feature_names = list(feature_names)

df = pd.DataFrame(list(zip(feature_names, clf.feature_importances_)))
df.columns = ['columns_name', 'score_feature_importance']
df.sort_values(by=['score_feature_importance'], ascending = False)